In [ ]:
import itertools as it
from collections import Counter, defaultdict
from functools import partial
from glob import glob
from pathlib import Path

import gfapy
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
pl.enable_string_cache()

# Functions

In [ ]:
def concat_glob(filename):
    return pl.concat([pl.scan_ipc(f) for f in glob(filename)], how="diagonal")

In [ ]:
def load_sequencing(filename, filter=True, path_col="variants_path"):
    df = concat_glob(filename)
    df = df.with_columns(
        dup=pl.col("name").is_duplicated(),
        primary_alignment=pl.col("name").is_first_distinct(),
    )
    if filter:
        df = df.filter(pl.col("primary_alignment"))
    df = df.with_columns(
        e2e=pl.col(path_col)
        .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
        .list.len()
        == 2,
        bc_e2e=pl.col(path_col)
        .list.set_intersection(
            ["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"]
        )
        .list.len()
        == 2,
    )
    if filter:
        df = df.filter(pl.col("e2e"))
    return df

# Analysis

In [ ]:
%%time
df = load_sequencing(
    "/home/jqs1/scratch/sequencing/240513_pLIB473_476/20240513_1645_2C_PAW46239_b49d575f/output/max_divergence=0.3/prepare_reads/*.arrow",
    path_col="full_path",
)  # .collect()

In [ ]:
%%time
df = concat_glob(
    "/home/jqs1/scratch/sequencing/240513_pLIB473_476/20240513_1645_2C_PAW46239_b49d575f/output/max_divergence=0.3/prepare_reads/*.arrow"
)
df = df.with_columns(
    # dup=pl.col("name").is_duplicated(),
    primary_alignment=pl.col("name").is_first_distinct(),
)
df = df.filter(pl.col("primary_alignment"))
df = df.filter(pl.col("path_hash") % 10000 == 17)
df = df.with_columns(
    e2e=pl.col("full_path")
    .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
    .list.len()
    == 2,
    bc_e2e=pl.col("full_path")
    .list.set_intersection(["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"])
    .list.len()
    == 2,
)
df = df.filter(pl.col("e2e"))
df = df.collect()

In [ ]:
0

In [ ]:
%%time
df.select(pl.len()).collect()

In [ ]:
%%time
df.select(pl.len()).collect()

In [ ]:
df.columns

In [ ]:
%%time
names = df.filter(pl.col("name").is_duplicated().not_()).select("name").collect()

In [ ]:
%%time
names = df.filter(pl.col("name").is_first_distinct()).select("name").collect()

In [ ]:
df2 = df.with_columns(
    # dup=pl.col("name").is_duplicated(),
    # primary_alignment=pl.col("name").is_first_distinct(),
    e2e=pl.col("full_path")
    .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
    .list.len()
    == 2,
    bc_e2e=pl.col("full_path")
    .list.set_intersection(["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"])
    .list.len()
    == 2,
)

In [ ]:
%%time
names = df2.filter(pl.col("name").is_first_distinct()).select("name").collect()

In [ ]:
%%time
names = df.filter(pl.col("primary_alignment")).select("name").collect()

In [ ]:
names

In [ ]:
%%time
paths_by_barcode = (
    df.filter(
        # pl.col("primary_alignment"), pl.col("e2e"), pl.col("path_hash") % 10000 == 17
        pl.col("path_hash") % 10000
        == 17
    )
    # .group_by("path_hash")
    # .agg(
    #     pl.col("path").first(),
    #     pl.struct("full_path", "reverse_complement").value_counts(),
    # )
    .collect()
)

In [ ]:
0

In [ ]:
paths_by_barcode

In [ ]:
%%time
x = paths_by_barcode.group_by("path_hash").agg(
    pl.col("path").first(),
    pl.struct("full_path", "reverse_complement").value_counts().alias("paths"),
)

In [ ]:
x.select(pl.col("paths").list.len()).sort("paths")

In [ ]:
%%time
paths_by_barcode = (
    df.filter(
        # pl.col("primary_alignment"), pl.col("e2e"), pl.col("path_hash") % 10000 == 0
        pl.col("path_hash") % 10000
        == 0
    )
    .group_by("path_hash")
    .agg(
        pl.col("path").first(),
        pl.struct("full_path", "reverse_complement").value_counts(),
    )
    .collect()
)

In [ ]:
df2 = df.head(1000).collect()

In [ ]:
df2

In [ ]:
df.head(100).collect()["path"]

In [ ]:
df.columns

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/sequencing/sequencing_references/pLIB473-476.gfa"
)

In [ ]:
df = df.with_columns(
    pl.col("extract_segments").struct.with_fields(
        pl.field("repressor:promoter|variant").replace({"pLacI": "pTac"})
    )
)

In [ ]:
df = processing.compute_divergences(
    df,
    list(dict.fromkeys(([s.split("=")[0] for s in gfa.segment_names]))),
    struct_name="extract_segments",
)
df2 = df.unnest("extract_segments")

In [ ]:
df2["repressor:promoter|variant"].value_counts()

In [ ]:
df2["repressor|variant"].value_counts()

In [ ]:
df2.group_by("repressor:promoter|variant", "repressor|variant").agg(pl.len()).sort(
    "len", descending=True
)

In [ ]:
df2.select(pl.col(r"^.*\|divergence$")).quantile(0.9).to_pandas().T.sort_values(0)

In [ ]:
df2 = df2.with_columns(
    max_divergence=pl.max_horizontal(
        pl.col(r"^.*\|divergence$").exclude(
            "term:DT3|divergence", "JUNC10_UNS10|divergence"
        )
    )
)

In [ ]:
plt.hist(df2["max_divergence"], bins=100, log=True);

In [ ]:
df2.filter(pl.col("max_divergence") > 0.01)

In [ ]:
df2.filter(pl.col("max_divergence") > 0.01).select(
    pl.col(r"^.*\|divergence$")
).quantile(0.9).to_pandas().T.sort_values(0)

In [ ]:
plt.hist(df2["max_divergence"], bins=100, log=True);

In [ ]:
counts = df2.select(
    pl.struct("RBS1:RBS|seq", "RBS2:RBS|seq", "RBS3:RBS|seq").alias("foo")
)["foo"].value_counts(sort=True)

In [ ]:
counts["count"].to_pandas().hvplot.step(logy=True, logx=True)

In [ ]:
df["grouping_depth"].sort(descending=True).to_pandas().hvplot.step(logy=True)

In [ ]:
counts.filter(pl.col("count") == 1)

In [ ]:
counts.filter(pl.col("count") > 5)

In [ ]:
with pl.Config(tbl_rows=100):
    display(counts)

In [ ]:
perfect_segments = [
    "pBetI",
    "RBS1:upstream",
    "LacI",
    "pTac",
    "RBS2:upstream",
    "PhlF",
    "pPhlF",
    "RBS3:upstream",
    "BetI",
]

In [ ]:
df2.filter(*[pl.col(f"{s}|divergence") == 0 for s in perfect_segments]).select(
    pl.struct("RBS1:RBS|seq", "RBS2:RBS|seq", "RBS3:RBS|seq").alias("foo")
)["foo"].value_counts(sort=True)

In [ ]:
filter_columns = [
    "promoter|divergence",
    "ScmJ|divergence",
    "RBS:RiboJ|divergence",
    "RBS:BCD_leader|divergence",
    "repressor:promoter|divergence",
    "repressor|divergence",
    "mVenus|divergence",
]

In [ ]:
157_002 / len(df2)

In [ ]:
df2.filter(*[pl.col(c) == 0 for c in filter_columns])

In [ ]:
%%time
cols = ["grouping_path", "variants_path", "grouping_depth"]
df_joined = df_240703.select(cols).join(
    df_240703b.select(cols),
    on=processing.categorical_list_hash(pl.col("grouping_path")),
    # on="grouping_path_hash",
    how="full",
    suffix="_b",
)